In [16]:
!pip install oauthlib
!pip install requests_oauthlib
!pip install sentinelhub

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/245.6 kB ? eta -:--:--
   ------------------------------------ --- 225.3/245.6 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 245.6/245.6 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
   ---------------------------------------- 137.6/137.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB 5.9 MB/s eta 0:00:03
   - -------------------------------------- 0.6/15.8 MB 5.9 MB/s 

In [13]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = 'sh-ce0eedb1-19f1-4a55-a1f9-a83e20a8e33c'
client_secret = 'VPmNjvnIF8TCcPKQ9Q6FyAEfxZbbktvq'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)


b'[]'


In [15]:
import requests

data = {
    "bbox": [17.995, 40.488, 17.262, 40.574],
    "datetime": "2024-04-07T00:00:00Z/2024-05-07T23:59:59Z",
    "collections": ["sentinel-1-grd", 'sentinel-2-l1c'],
    "limit": 5,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/search"
response = oauth.post(url, json=data)
response.json()

{'type': 'FeatureCollection',
 'features': [{'stac_version': '1.0.0',
   'stac_extensions': ['https://stac-extensions.github.io/sat/v1.0.0/schema.json',
    'https://stac-extensions.github.io/sar/v1.0.0/schema.json',
    'https://docs.sentinel-hub.com/api/latest/stac/s1/v1.0.0/schema.json'],
   'id': 'S1A_IW_GRDH_1SDV_20240502T164113_20240502T164138_053695_0685B8_F678_COG.SAFE',
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'crs': {'type': 'name',
     'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}},
    'coordinates': [[[17.524601314021922, 40.080611297803166],
      [17.94631330190608, 40.141965872165486],
      [18.252952856114796, 40.18553377483311],
      [18.711217706471025, 40.24903513869423],
      [19.01806486335668, 40.290476828651705],
      [19.479366498418674, 40.351157217872746],
      [19.795960406018647, 40.39166720865808],
      [20.101987540767933, 40.42997141022998],
      [20.442269146379054, 40.471538386709085],
      [20.40994349504167, 40.652

In [38]:
from sentinelhub import SHConfig, MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest
config = SHConfig()
config.instance_id = '880e1fdd-9935-4a80-b200-0c1468b320a8' 
config.sh_client_secret = 'F04HuxBiCLxleKFhsTQtxXddFpx1eYo2'
config.sh_client_id = '0a9b1bb0-53eb-4a7e-b340-332a3b5c70ba'

In [64]:
# Define the coordinates of the point of interest
point_coords_wgs84 = [17.995, 40.488]
padding = 0.2
# Define a small bounding box around the point of interest
bbox_coords_wgs84 = [point_coords_wgs84[0] - padding, point_coords_wgs84[1] - padding, point_coords_wgs84[0] + padding, point_coords_wgs84[1] + padding]

# Create a BBox instance
bbox = BBox(bbox_coords_wgs84, crs=CRS.WGS84)

# Define the size of the image
x_size = 200
_, y_size = bbox_to_dimensions(bbox, resolution=x_size)  # Extract height from the tuple

# Define the request
request = SentinelHubRequest(
    evalscript="""
        //VERSION=3
        function setup() {
            return {
                input: ["B02", "B03", "B04"],
                output: { bands: 3 }
            };
        }
        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
    """,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=('2024-04-07T00:00:00Z', '2024-05-07T23:59:59Z'),
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.JPG)
    ],
    bbox=bbox,
    size=(x_size, y_size),  # Pass height as a single integer
    config=config
)

# Download the image
image = request.get_data()[0]

# Display the image
from PIL import Image
print(image)
#Image.fromarray(image)

[[[44 42 45]
  [23 21 24]
  [26 26 28]
  ...
  [15 19 28]
  [15 19 28]
  [15 19 28]]

 [[38 36 39]
  [26 24 27]
  [25 25 27]
  ...
  [15 19 28]
  [15 19 28]
  [15 19 28]]

 [[27 25 28]
  [25 23 26]
  [21 21 23]
  ...
  [15 19 28]
  [15 19 28]
  [15 19 28]]

 ...

 [[47 48 50]
  [31 32 34]
  [25 26 28]
  ...
  [29 31 30]
  [28 30 29]
  [28 30 29]]

 [[35 36 38]
  [21 22 24]
  [18 19 21]
  ...
  [33 33 33]
  [31 33 32]
  [30 32 31]]

 [[40 41 43]
  [40 41 43]
  [38 39 41]
  ...
  [29 29 29]
  [30 32 31]
  [31 33 32]]]
